## How to access a Python variable in PAN

In [ ]:
import os
import pypan.ui as pan
from pypan.utils import load_vars
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

The purpose of this notebook is to illustrate how a Python variable can be used in a PAN netlist.

As an example, we will use the Van der Pol oscillator: for a detailed description of the model, see the [Van der Pol notebook](Van_der_Pol.ipynb).

The following netlist defines the Van der Pol circuit in PAN. Notice the presence of the <tt>port</tt> device, whose parameter <tt>noisesamples</tt> specifies the name of the variable containing the noisy samples. The content of this variable will be explained in the following. 

In [ ]:
netlist = """\
ground electrical gnd

parameters G=1/10k

; Circuit

p1    x   gnd  x    gnd  poly n0=0 n1=-1 n2=0 n3=1 d0=1
c1    x   gnd  capacitor c=1 icon=200m
rx    x   gnd  RES  r=1/G
l1    x   gnd  inductor  l=1

ns    x   gns  port noisesamples="noisy_py"

model RES resistor kf=0 af=1
"""

#### Write the above netlist to file

In [ ]:
netlist_file = 'noisy_vanderpol.pan'
with open(netlist_file, 'w') as fid:
    fid.write(netlist)

### Load the netlist with the description of the circuit
No analysis is performed at this point, since none is defined in the netlist.

In [ ]:
ok,libs = pan.load_netlist(netlist_file)
if not ok:
    print('load_netlist failed.')

### Build the variable to be used in PAN
The noise samples need to be stored in a matrix with two columns, representing time instants and noise values, respectively. The number of samples needs to be sufficient to cover the whole duration of the simulation: PAN will use the same noise value for those time instants that fall in between the ones provided by the user.

The samples matrix can be implemented in two ways:
1. an N-by-2 NumPy array, where N is the number of samples.
1. a two-element list, where each element of the list is itself another list with N elements.

The variable <tt>use_numpy</tt> below allows choosing between these two options.

In [ ]:
use_numpy = True
dt = 0.1
tstop = 100
if use_numpy:
    t = np.r_[0 : tstop + 2*dt : dt]
    noisy_py = np.zeros((len(t), 2))
    noisy_py[:,0] = t
    noisy_py[:,1] = np.random.normal(0, 10, size=(len(t),))
else:
    t = [i * dt for i in range(int(tstop/dt) + 1)]
    t.append(t[-1] + dt)
    rnd = [np.random.uniform() for _ in range(len(t))]
    noisy_py = [t, rnd]

### Run a transient analysis

Notice that the <tt>noiseinj=2</tt> option to the tran analysis specifies that noise is injected only in the (nonlinear) circuit and therefore affects exclusively the large signal solution. <br/>
Alternatively, one could use <tt>noiseinj=1</tt> to indicate that noise should be injected in the variational part of the circuit.

In [ ]:
tran_name = 'Tr'
mem_vars = ['time', 'x', 'l1:i']
time,vc,il = pan.tran(tran_name, tstop, mem_vars, libs, uic=1, noisefmax=10, noiseinj=2)

### Load the same variables from disk
This is an alternative to calling the <tt>tran</tt> method with the <tt>mem_vars</tt> argument. <br/>
The same is valid for all the other methods in <tt>pypan</tt> that accept a <tt>mem_vars</tt> argument.

In [ ]:
data = load_vars(os.path.splitext(netlist_file)[0] + '.raw/' + tran_name, mem_vars)

### Plot the results

In [ ]:
fig,(ax1,ax2) = plt.subplots(1, 2, sharex=True, figsize=(12,5))
    
ax1.plot(data['time'], data['x'], 'r', linewidth=1, label='From disk')
ax1.plot(time, vc, 'k', linewidth=1, label='From memory')
ax1.set_xlabel('Time [s]')
ax1.set_ylabel(r'$v_C$ [V]')
ax1.legend(loc='best')
    
ax2.plot(data['time'], data['l1:i'], 'r', linewidth=1, label='From disk')
ax2.plot(time, il, 'k', linewidth=1, label='From memory')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel(r'$i_L$ [A]');